# Deep Learning
# DL02 Tensores en Pytorch



En este notebook, presentamos  [PyTorch] (http://pytorch.org/), un framework para construir y entrenar redes neuronales. En el calculo de redes neuronales se utilizan frecuentemente matrices y de forma mas general tensores.  PyTorch toma estos tensores y hace que sea simple moverlos a GPU para el procesamiento más rápido necesario al entrenar redes neuronales. También proporciona un módulo que calcula automáticamente los gradientes (¡para propagación hacia atrás!) Y otro módulo específicamente para construir redes neuronales. 

## <font color='blue'>**Redes Neuronales.**</font>

Deep Learning se basa en redes neuronales artificiales que han existido de alguna forma desde finales de la década de 1950. Las redes se construyen a partir de partes individuales que se aproximan a las neuronas, típicamente llamadas unidades o simplemente "neuronas". Cada unidad tiene un cierto número de entradas ponderadas. Estas entradas ponderadas se suman (una combinación lineal) y luego se pasan a través de una función de activación para obtener la salida de la unidad.

![Log](https://drive.google.com/uc?export=view&id=1EBHN-Ho1ZmYoRy1x2ZkER9fLWBSTwvO3) 


Matematicamente esto se ve de la siguiente forma:

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

Si lo expereamos en notacion vectorial esto es basicamente un prodcuto interno entre dos vectores.

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

### Tensores la base de los calculos en redes neuronales. 

Resulta que los cálculos de la red neuronal son solo un montón de operaciones de álgebra lineal de **tensores** (una generalización de las matrices). Un vector es un tensor unidimensional, una matriz es un tensor bidimensional, una matriz con tres índices es un tensor tridimensional (imágenes de color RGB, por ejemplo). La estructura de datos fundamental para las redes neuronales son los tensores y PyTorch (así como casi cualquier otro framework de aprendizaje profundo) se construye alrededor de los tensores.


![Log](https://drive.google.com/uc?export=view&id=1p_zNdbAbwDPCk4ZvZ_2U7MklfkW8OCr1) 




### Exploremos cómo podemos usar PyTorch para construir una red neuronal simple.

In [ ]:
# Primero importemos PyTorch
import torch

In [ ]:
# Construyamos 
def activation(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [ ]:
### Generemos datos.
torch.manual_seed(7) # Definamos una semilla para poder reproducir el calculo.

# Features son 5 variables normales aleatorias. 
features = torch.randn((1, 5))
print(features)
# Retorna pesos utilizando una distribución normal con media 0 y variana 1.
weights = torch.randn_like(features)
print(weights)
#  Adicionalmente definimos un bias. 
bias = torch.randn((1, 1))

tensor([[-0.1468,  0.7861,  0.9468, -1.1143,  1.6908]])
tensor([[-0.8948, -0.3556,  1.2324,  0.1382, -1.6822]])


Arriba se generaron  datos que podemos usar para obtener la salida de nuestra red simple. Todo esto es aleatorio por ahora, en adelante comenzaremos a usar datos normales. Analisemos cada linea:

`features = torch.randn((1, 5))` crea un tensor de forma `(1, 5)`, 1 fila y 5 columnas, este vector contiene valores aleatoriamente distribuidos de acuerdo a una distribución normal con media 0 y desviación estandar 1.

`weights = torch.randn_like(features)` crea otro tensor con la misma forma que `features`, y nuevamente conteniendo valores de una distribución normal. 

Finalmente , `bias = torch.randn((1, 1))` crea un unico valor obtenido de una distribución normal. 

Los tensores PyTorch se pueden agregar, multiplicar, restar, etc., al igual que las matrices de Numpy. En general, usará los tensores PyTorch de la misma manera que usaría las matrices Numpy. Sin embargo, vienen con algunos buenos beneficios, como la aceleración de GPU, que veremos más adelante. Por ahora, use los datos generados para calcular la salida de esta red simple de una sola capa.


**Ejemplo**: Calcule la salida de la redo considerando como input a `features`, como pesos `weights`, y bias `bias`. PyTorch tiene un metodo [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum), para tomar sumas. Adicionalmente utilice la función `activation` definida  anteriormente como función de activación. 

In [ ]:
### Solucion
# Sol 1
y = activation(torch.sum(features * weights) + bias)
print(y)
# Sol 2
y = activation((features * weights).sum() + bias)
print(y)

tensor([[0.1595]])
tensor([[0.1595]])


Sin embargo, puedes hacer la multiplicación y la suma en la misma operación usando una multiplicación matricial. En general, querrás usar multiplicaciones matriciales ya que son más eficientes y aceleradas usando bibliotecas modernas y computación de alto rendimiento en GPU.

Aquí, queremos hacer una multiplicación matricial de las features y los weights. Para esto podemos usar [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) or [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul)

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```
A medida que construye redes neuronales en cualquier framework, este error aparecerá a menudo. Lo que sucede aquí es que nuestros tensores no son las formas correctas para realizar una multiplicación matricial. Recuerde que para las multiplicaciones matriciales, El número de columnas en el primer tensor debe ser igual al número de filas en la segunda columna. Ambas `features` y` weights` tienen la misma forma, `(1, 5)`. Esto significa que necesitamos cambiar la forma de los 'pesos' para que la multiplicación de la matriz funcione.


**Nota:** Para ver la forma de un tensor `tensor`, usamos `tensor.shape`. 

Ecisten opciones para cambiar la forma a un tensor: [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_), and [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

* `weights.reshape(a, b)` Retornará un tensor con la misma data que `weights` y con tamaño `(a, b)`. 
* `weights.resize_(a, b)` Retorna un tensor con distinta forma. Si la nueva forma tiene menos elementos que la original, algunos seran removidos. Si la nueva forma tiene mas elementos que el original, los nuevos elementos serán no inicializados.   [read more about in-place operations](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) in PyTorch.
* `weights.view(a, b)` retornará el mismo tensor con la misma data que  `weights` con tamaño `(a, b)`.

> **Ejemplo**: Calcule la salida de nuestra pequeña red usando la multiplicación de matrices.

In [ ]:
## Solucion

y = activation(torch.mm(features, weights.view(5,1)) + bias)
print(y)

tensor([[0.1595]])


### ¡Apilarlos!

Así es como puede calcular la salida de una sola neurona. El poder real de este algoritmo ocurre cuando comienzas a apilar estas unidades individuales en capas y pilas de capas, en una red de neuronas. La salida de una capa de neuronas se convierte en la entrada para la siguiente capa. Con múltiples unidades de entrada y unidades de salida, ahora necesitamos expresar los pesos como una matriz.

![Log](https://drive.google.com/uc?export=view&id=1baAB8q9xxML3osQFAHfWdQQPzpMTEf-G) 


La primera capa que se muestra en la parte inferior aquí son las entradas, llamadas ** capa de entrada **. La capa intermedia se llama ** capa oculta **, y la capa final (a la derecha) es la ** capa de salida **. Podemos expresar esta red matemáticamente con matrices nuevamente y usar la multiplicación de matrices para obtener combinaciones lineales para cada unidad en una operación. Por ejemplo, la capa oculta ($ h_1 $ y $ h_2 $ aquí) se puede calcular

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

La salida para esta pequeña red se encuentra tratando la capa oculta como entradas para la unidad de salida. La salida de la red se expresa simplemente


$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

## <font color='green'>**Actividad 1**</font>

1. Defina una semilla para reproducir el calculo

2. Genere un dataset  con tres variables aleatorias.

3. Defina el tensor de pesos aleatorios entre la capa de entrada y la capa hidden. 

4. Defina un tensor de pesos aleatorios entre la capa hidden y la de salida.

5. Defina los tensores bias para la capa hidden y de salida.

6. Calcule la salida para esta red multicapa utilizando los pesos `W1` y` W2`, y los sesgos, `B1` y` B2`.




```
torch.mm(features, W1)
```



In [ ]:
### Construya los tensores y un dataset de prueba.
torch.manual_seed(7) # Asignemos una semilla para poder reproducir el calculo. 

In [30]:
### Enlace las distintas capas. Utilice torch.mm para realizar la multiplicación de tensores. 
"""
Si fijó la semilla en 7 el resultado debiese ser:
tensor([[0.6813, 0.4355]])
tensor([[0.3171]])
"""
import torch.nn.functional as F

# setting seed
torch.manual_seed(7)

# generating some random features
features = torch.randn(1, 3) 

# define the weights
W1 = torch.randn((3, 2), requires_grad=True)
W2 = torch.randn((2, ), requires_grad=True)

# define the bias terms
B1 = torch.randn((2), requires_grad=True)
B2 = torch.randn((), requires_grad=True)

# calculate hidden and output layers
h1 = activation(torch.mm(features, W1) + B1)
output = torch.sigmoid((h1 @ W2) + B2)

print(f"Atributos:\n{features}\n\n")
print(f"Vector de pesos capa de entrada:\n{W1}\n\n")
print(f"Vector de pesos capa oculta:\n{W2}\n\n")
print(f"Vector de bias para capa oculta:\n{B1}\n\n")
print(f"Vector de bias para capa de salida:\n{B2}\n\n")
print(f"Vector de outputs capa oculta:\n{h1}")
print(f"Resultado final:\n{output}\n\n")

Atributos:
tensor([[-0.1468,  0.7861,  0.9468]])


Vector de pesos capa de entrada:
tensor([[-1.1143,  1.6908],
        [-0.8948, -0.3556],
        [ 1.2324,  0.1382]], requires_grad=True)


Vector de pesos capa oculta:
tensor([-1.6822,  0.3177], requires_grad=True)


Vector de bias para capa oculta:
tensor([0.1328, 0.1373], requires_grad=True)


Vector de bias para capa de salida:
0.2405461221933365


Vector de outputs capa oculta:
tensor([[0.6813, 0.4355]], grad_fn=<MulBackward0>)
Resultado final:
tensor([0.3171], grad_fn=<SigmoidBackward0>)




<font color='green'>**Fin Actividad 1**</font>

Si hizo esto correctamente, debería ver la salida `tensor ([[0.3171]])`.

El número de unidades ocultas de un parámetro de la red, a menudo llamado **hiperparámetro** para diferenciarlo de los parámetros de weight y bias. Como verá más adelante cuando analicemos cómo entrenar una red neuronal, cuantas más unidades ocultas tenga una red y más capas, mejor podrá aprender de los datos y hacer predicciones precisas.